In [31]:
# load module and pre generate data
from graph_tree_list_module import *

import numpy as np

import pickle

# Open the file in binary read mode and load the object
with open("graph_tree.pkl", "rb") as file:
    gt = pickle.load(file)


In [32]:
sliders_data = [{"id": gt.attr_name[i], "min": gt.X_maxmin[1][i], "max": gt.X_maxmin[0][i], "value": gt.nodes[0].center[i], "type": "f"} for i in range(len(gt.attr_name))]
# print(sliders_data)

recom_data = [{"i": i, "id": gt.attr_name[i], "max_val": gt.recom[i][0].center[i], "min_val": gt.recom[i][1].center[i]} for i in range(len(gt.attr_name))]
# print(recom_data)

graph_tree_data = gt.get_neighbour_cluster_data(node_id = 85)

tree_data = {
        "name": "Root",
        "children": [
            {
                "name": "Child 1",
                "children": [
                    {"name": "Grandchild 1"},
                    {"name": "Grandchild 2"}
                ]
            },
            {
                "name": "Child 2",
                "children": [
                    {"name": "Grandchild 3"},
                    {"name": "Grandchild 4"}
                ]
            }
        ]
    }

reflection_node: 11 at lv up: 2
generate cos sim, ecul dist, and nodes sorted by cos sim


In [30]:
from flask import Flask, render_template, request, jsonify

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

# slider ####################################################################
@app.route('/sliders')
def sliders():
    return jsonify(sliders_data)

@app.route('/handle_slider', methods=['POST'])
def submit_slider_values():
    data = request.json
    print('Received slider values:', data)
    query = np.array([float(val) for val in data.values()])
    cur = gt.search(query)
    graph_tree_data = gt.get_neighbour_cluster_data(cur.id)
    # Process the data as needed
    return jsonify({'status': 'slider respon success', 'graph_tree_data': graph_tree_data})
# slider end ################################################################

# recom ####################################################################
@app.route('/recom')
def recom():
    return jsonify(recom_data)

@app.route('/handle_recom', methods=['POST'])
def handle_recom():
    data = request.json
    button_id = int(data.get('buttonId'))
    val = int(data.get('val'))
    # print(button_id, val)
    moved_node = gt.recom[button_id][val]
    graph_tree_data = gt.get_neighbour_cluster_data(moved_node.id)
    # print(graph_tree_data)
    return jsonify({'status': 'recom respon success', 'buttonId': button_id, 'val': val, "graph_tree_data": graph_tree_data})
# recom end ####################################################################

@app.route('/graph_tree_data')
def graph():
    return jsonify(graph_tree_data)

@app.route('/graph_tree_circle_click', methods=['POST'])
def graph_tree_circle_click():
    data = request.json
    id = int(data.get('id'))
    print(id)
    graph_tree_data = gt.get_neighbour_cluster_data(id)
    return jsonify({'status': 'graph_tree_circle_click respon success', 'id': id, "graph_tree_data": graph_tree_data})


@app.route('/tree_data')
def tree():
    return jsonify(tree_data)

if __name__ == '__main__':
    app.run(port=8123)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8123
Press CTRL+C to quit
127.0.0.1 - - [03/Oct/2024 21:17:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Oct/2024 21:17:31] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Oct/2024 21:17:31] "GET /static/scripts.js HTTP/1.1" 304 -
127.0.0.1 - - [03/Oct/2024 21:17:31] "GET /static/curve_axis.js HTTP/1.1" 304 -
127.0.0.1 - - [03/Oct/2024 21:17:31] "GET /static/graph_tree.js HTTP/1.1" 304 -
127.0.0.1 - - [03/Oct/2024 21:17:31] "GET /sliders HTTP/1.1" 200 -
127.0.0.1 - - [03/Oct/2024 21:17:31] "GET /recom HTTP/1.1" 200 -
127.0.0.1 - - [03/Oct/2024 21:17:31] "GET /graph_tree_data HTTP/1.1" 200 -
127.0.0.1 - - [03/Oct/2024 21:17:31] "GET /tree_data HTTP/1.1" 200 -
127.0.0.1 - - [03/Oct/2024 21:17:35] "POST /handle_recom HTTP/1.1" 200 -


11 0
reflection_node: 1 at lv up: 2
generate cos sim, ecul dist, and nodes sorted by cos sim
[{'id': 20, 'cos_sim': np.float64(-0.5141220218543034), 'euclide_dist': np.float64(1.0925226838798079), 'r_ratio': 1.8333333333333333}, {'id': 17, 'cos_sim': np.float64(-0.5099567531398856), 'euclide_dist': np.float64(0.8027512371395716), 'r_ratio': 7.25}, {'id': 21, 'cos_sim': np.float64(-0.2858940884192117), 'euclide_dist': np.float64(1.0015505951762458), 'r_ratio': 1.75}, {'id': 19, 'cos_sim': np.float64(-0.27765299941336974), 'euclide_dist': np.float64(0.9818749707130934), 'r_ratio': 1.9166666666666667}, {'id': 16, 'cos_sim': np.float64(-0.2114062464791594), 'euclide_dist': np.float64(0.7128433332564057), 'r_ratio': 5.833333333333333}, {'id': 6, 'cos_sim': np.float64(-0.08046292069716708), 'euclide_dist': np.float64(1.2005655131096171), 'r_ratio': 2.9166666666666665}, {'id': 18, 'cos_sim': np.float64(-0.06110821408248549), 'euclide_dist': np.float64(0.7097882330366457), 'r_ratio': 8.5833333

127.0.0.1 - - [03/Oct/2024 21:17:36] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Oct/2024 21:20:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Oct/2024 21:20:00] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Oct/2024 21:20:00] "GET /static/scripts.js HTTP/1.1" 304 -
127.0.0.1 - - [03/Oct/2024 21:20:00] "GET /static/curve_axis.js HTTP/1.1" 304 -
127.0.0.1 - - [03/Oct/2024 21:20:00] "GET /static/graph_tree.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Oct/2024 21:20:00] "GET /sliders HTTP/1.1" 200 -
127.0.0.1 - - [03/Oct/2024 21:20:00] "GET /recom HTTP/1.1" 200 -
127.0.0.1 - - [03/Oct/2024 21:20:00] "GET /graph_tree_data HTTP/1.1" 200 -
127.0.0.1 - - [03/Oct/2024 21:20:00] "GET /tree_data HTTP/1.1" 200 -
127.0.0.1 - - [03/Oct/2024 21:20:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Oct/2024 21:20:15] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Oct/2024 21:20:15] "GET /static/scripts.js HTTP/1.1" 304 -
127.0.0.1 - - [03/Oct/2024 21:20:15] "GET /static/curve_axi

11 0
reflection_node: 1 at lv up: 2
generate cos sim, ecul dist, and nodes sorted by cos sim
[{'id': 20, 'cos_sim': np.float64(-0.5141220218543034), 'euclide_dist': np.float64(1.0925226838798079), 'r_ratio': 1.8333333333333333}, {'id': 17, 'cos_sim': np.float64(-0.5099567531398856), 'euclide_dist': np.float64(0.8027512371395716), 'r_ratio': 7.25}, {'id': 21, 'cos_sim': np.float64(-0.2858940884192117), 'euclide_dist': np.float64(1.0015505951762458), 'r_ratio': 1.75}, {'id': 19, 'cos_sim': np.float64(-0.27765299941336974), 'euclide_dist': np.float64(0.9818749707130934), 'r_ratio': 1.9166666666666667}, {'id': 16, 'cos_sim': np.float64(-0.2114062464791594), 'euclide_dist': np.float64(0.7128433332564057), 'r_ratio': 5.833333333333333}, {'id': 6, 'cos_sim': np.float64(-0.08046292069716708), 'euclide_dist': np.float64(1.2005655131096171), 'r_ratio': 2.9166666666666665}, {'id': 18, 'cos_sim': np.float64(-0.06110821408248549), 'euclide_dist': np.float64(0.7097882330366457), 'r_ratio': 8.5833333

...

0 1
reflection_node: 11 at lv up: 2
generate cos sim, ecul dist, and nodes sorted by cos sim
[{'id': 75, 'cos_sim': np.float64(-0.40146584506070443), 'euclide_dist': np.float64(1.236891236877308), 'r_ratio': 39.0}, {'id': 78, 'cos_sim': np.float64(-0.3001024550950954), 'euclide_dist': np.float64(1.2929076612759856), 'r_ratio': 6.0}, {'id': 82, 'cos_sim': np.float64(-0.08142391463730748), 'euclide_dist': np.float64(1.3526893311215873), 'r_ratio': 3.0}, {'id': 81, 'cos_sim': np.float64(-0.055923329846542566), 'euclide_dist': np.float64(1.1461230376675957), 'r_ratio': 13.0}, {'id': 76, 'cos_sim': np.float64(-0.030370417473648294), 'euclide_dist': np.float64(1.15410287837338), 'r_ratio': 38.0}, {'id': 79, 'cos_sim': np.float64(-0.023275017564249125), 'euclide_dist': np.float64(1.1565406793676372), 'r_ratio': 6.0}, {'id': 85, 'cos_sim': np.float64(0.10696218686052496), 'euclide_dist': np.float64(1.2223322428629215), 'r_ratio': 2.0}, {'id': 77, 'cos_sim': np.float64(0.1257466865862496), 'euc

127.0.0.1 - - [03/Oct/2024 23:20:36] "POST /handle_recom HTTP/1.1" 200 -


0 0
reflection_node: 3 at lv up: 2
generate cos sim, ecul dist, and nodes sorted by cos sim
[{'id': 48, 'cos_sim': np.float64(-0.7881104338744571), 'euclide_dist': np.float64(0.7660269906594852), 'r_ratio': 6.0}, {'id': 49, 'cos_sim': np.float64(-0.5755837436160911), 'euclide_dist': np.float64(0.821815261591754), 'r_ratio': 5.0}, {'id': 50, 'cos_sim': np.float64(-0.45745308276831026), 'euclide_dist': np.float64(0.835136823283391), 'r_ratio': 2.8}, {'id': 47, 'cos_sim': np.float64(-0.07807131362109994), 'euclide_dist': np.float64(0.6731499997258792), 'r_ratio': 4.6}, {'id': 56, 'cos_sim': np.float64(0.10043949592213013), 'euclide_dist': np.float64(0.8164680996364002), 'r_ratio': 0.6}, {'id': 52, 'cos_sim': np.float64(0.20124614163727422), 'euclide_dist': np.float64(0.5881525934492925), 'r_ratio': 1.6}, {'id': 54, 'cos_sim': np.float64(0.31029943494230827), 'euclide_dist': np.float64(0.6637400211753084), 'r_ratio': 1.0}, {'id': 53, 'cos_sim': np.float64(0.3822019542350906), 'euclide_dist

127.0.0.1 - - [04/Oct/2024 00:22:46] "POST /handle_recom HTTP/1.1" 200 -


13 0
reflection_node: 3 at lv up: 2
generate cos sim, ecul dist, and nodes sorted by cos sim
[{'id': 49, 'cos_sim': np.float64(-0.5159931062205891), 'euclide_dist': np.float64(0.811417303426646), 'r_ratio': 5.0}, {'id': 52, 'cos_sim': np.float64(-0.43880974678194795), 'euclide_dist': np.float64(0.7627067071768198), 'r_ratio': 1.6}, {'id': 50, 'cos_sim': np.float64(-0.35568852474528356), 'euclide_dist': np.float64(0.8105352579768698), 'r_ratio': 2.8}, {'id': 58, 'cos_sim': np.float64(-0.2286117642974995), 'euclide_dist': np.float64(0.8014764138101276), 'r_ratio': 1.6}, {'id': 47, 'cos_sim': np.float64(-0.1315458787060878), 'euclide_dist': np.float64(0.6917515672437081), 'r_ratio': 4.6}, {'id': 48, 'cos_sim': np.float64(0.024155584346407667), 'euclide_dist': np.float64(0.6106281746973509), 'r_ratio': 6.0}, {'id': 53, 'cos_sim': np.float64(0.1259118111097981), 'euclide_dist': np.float64(0.6306872596442936), 'r_ratio': 2.8}, {'id': 55, 'cos_sim': np.float64(0.21117179945530268), 'euclide_d